# Movie Recommendation System

## Loading movie data

In [ ]:
data_dir: str = "data"

In [ ]:
import os

Define the CSV file of the movie table

In [ ]:
movies_csv: str = os.path.join(data_dir, "movies.csv")

In [ ]:
import numpy as np
from typing import Type

Define the dtype for minimum required space

In [ ]:
movies_dtype: dict[str, Type] = {
    "movieId": np.int32,
    "title": str,
    "genres": str
}

In [ ]:
import pandas as pd

Load the CSV into a dataframe

In [ ]:
movies_df: pd.DataFrame = pd.read_csv(movies_csv, dtype=movies_dtype)

In [ ]:
movies_df.head()

## Cleaning movie data

Drop the rows with missing values

In [ ]:
movies_clean: pd.DataFrame = movies_df.dropna()

In [ ]:
movies_df.shape[0], movies_clean.shape[0]

## Genre one-hot encoding

Get a dataframe of just the `movieId` and `genres`

In [ ]:
movies_feat: pd.DataFrame = movies_clean.copy()
movies_feat = movies_feat.drop(columns=["title"])

Turn the `genres` column into a list of strings

In [ ]:
movies_feat["genres"] = movies_feat["genres"].str.split("|")

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

Fit transform the `genres` column into one-hot encodings

In [ ]:
mlb: MultiLabelBinarizer = MultiLabelBinarizer()
mlb.fit_transform(movies_feat["genres"])

In [ ]:
cache_dir: str = "cache"

Define the JSON file to save the MLB classes into

In [ ]:
mlb_classes_json: str = os.path.join(cache_dir, "mlb_classes.json")

In [ ]:
import json

Cache the MLB classes into a JSON file for re-use

In [ ]:
with open(mlb_classes_json, "w") as f:
    json.dump(mlb.classes_.tolist(), f)

Load the MLB with the cached classes to check if it works

In [ ]:
with open(mlb_classes_json, "r") as f:
    mlb = MultiLabelBinarizer(classes=json.load(f))

In [ ]:
from scipy.sparse import spmatrix
from typing import Union
from numpy.typing import NDArray

Get the one-hot encodings

In [ ]:
encodings: Union[NDArray[np.int32], spmatrix] = mlb.fit_transform(
    movies_feat["genres"])

Store the one-hot encodings as an integer as there are less than 32 genres in the dataframe

In [ ]:
genre_bits: NDArray[np.int32] = np.zeros(len(movies_feat), dtype=np.int32)

for i in range(encodings.shape[1]):
    genre_bits = genre_bits | (encodings[:, i].astype(np.int32) << i)

In [ ]:
movies_feat["genres"] = genre_bits

In [ ]:
movies_feat.head()

## User Input

Define the CSV file containing title and rating

In [ ]:
input_csv: str = os.path.join(data_dir, "input.csv")

Load the inputs into a dataframe

In [ ]:
input_df: pd.DataFrame = pd.read_csv(input_csv)

In [ ]:
input_df.head()

## Input fuzzy matching

In [ ]:
from rapidfuzz import process
from typing import Any

Get the `movieId` by doing a fuzzy search on the title

In [ ]:
input_feat_list: list[dict[str, Any]] = []

for _, row in input_df.iterrows():
    matched: tuple[str, float, int] = process.extractOne(row["title"],
                                                         movies_clean["title"])
    if not matched or not matched[0]:
        raise Exception(f"Invalid match for {row['title']}: {matched[0]}")

    movie_row = movies_clean[movies_clean["title"] == matched[0]].iloc[0]

    confirmation_prompt = (f"Got '{movie_row['title']}' for '{row['title']}'. "
                           f"Add? (Y/n)?")
    confirmation = input(confirmation_prompt) or "y"

    match confirmation.lower():
        case "y" | "yes":
            input_feat_list.append({
                "movieId": movie_row["movieId"],
                "rating": row["rating"]
            })

        case "n" | "no" | _:
            pass

In [ ]:
input_feat = pd.DataFrame(input_feat_list)

In [ ]:
input_feat.head()

Define the CSV to save the input features

In [ ]:
input_feat_csv: str = os.path.join(cache_dir, "input_feat.csv")

Save the input features to CSV

In [ ]:
input_feat.to_csv(input_feat_csv, index=False)

Load the input features from cache to check if it works

In [ ]:
input_feat = pd.read_csv(input_feat_csv)

In [ ]:
input_feat.head()

## Content-based filtering

In [ ]:
import numba as nb

Use `numba` for fast performance

In [ ]:
@nb.njit
def popcount(x):
    count = 0
    while x:
        count += x & 1
        x >>= 1
    return count

Pre-computer movie-genre mapping

In [ ]:
movie_genres = dict(zip(movies_feat["movieId"], movies_feat["genres"]))

Calculate the sum similarity score across all input movies

In [ ]:
similarity_scores = {}

for _, row in input_feat.iterrows():
    source_genres = movie_genres[row["movieId"]]

    for mov_id, target_genres in movie_genres.items():
        common = source_genres & target_genres
        extra = target_genres & ~source_genres

        sim_score = popcount(common) - popcount(extra)
        similarity_scores[mov_id] = similarity_scores.get(mov_id, 0) + sim_score

In [ ]:
dict(list(similarity_scores.items())[:10])

## Define the CSV loader

In [ ]:
from pandas.io.parsers import TextFileReader
from typing import Generator, Type

Read the CSV in chunks to optimize memory utilization

In [ ]:
def read_csv_in_chunks(
    file_path: str,
    dtype: dict[str, Type],
    chunk_size: int = 100_000
) -> Generator[pd.DataFrame, None, None]:
    reader: TextFileReader = pd.read_csv(file_path, chunksize=chunk_size,
                                         dtype=dtype)
    for chunk in reader:
        chunk: pd.DataFrame
        yield chunk.dropna()

## Collabarative filtering